In [57]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from bs4 import element
from time import sleep
import pandas as pd


def get_page_data(year):
    url = 'http://www.sports-reference.com/cbb/postseason/'+str(year)+'-ncaa.html'
    page = urlopen(url)
    return BeautifulSoup(page, 'html.parser')

def tidy_location(loc_str):
    if loc_str.startswith('at '):
        return loc_str[len('at '):]

def tidy_team_name(team_name):
    team_name.replace(';','')

In [59]:
def get_bracket():
    # this works for 84 first round bye and 85 and later 64+ team bracket ignoring play-in games
    years = range(2016, 1938, -1)
    bracket = []
    for year in years: 
        soup = get_page_data(year)
        region_tags = soup.find_all("div", attrs={"data-controls": "#brackets"})[0].select('div a')           
        #print(year)
        for region_tag in region_tags:        
            region = str(region_tag.string).lower()                           
            if region == 'final four':
                region_div = soup.find('div', {'id':'national'})
            else:
                search = {'id': region} 
                region_div = soup.find('div',**search)            
            if region_div is not None:
                bracket_rounds = region_div.div
                div_rounds = bracket_rounds.find_all('div', {'class':'round'})                
                if region == 'final four':
                    bracket_round = 5
                else:
                    bracket_round = 1
                for div_round in div_rounds:
                    #print("round, ", bracket_round)
                    if isinstance(div_round, element.Tag ):
                        #print(len(div_round))
                        for gm in div_round:
                            if isinstance(gm, element.Tag ):
                                #print(gm)
                                #print("len contents: ", len(gm.contents))
                                #print()
                                if (len(gm.contents) >= 9):
                                    #print(gm)
                                    #print()
                                    seeds_loc = gm.find_all('span')
                                    if len(seeds_loc) > 2:                                        
                                        winner = gm.find('div', {'class':'winner'})
                                        #if winner is None:
                                        #    print(winner)
                                        #else:
                                        winner_seed = winner.find('span').string                                        
                                        seed1 = seeds_loc[0].string                                        
                                        seed2 = seeds_loc[1].string                                        
                                        currLoc = tidy_location(seeds_loc[2].string)        
                                        alist = gm.select('div a')                                        
                                        #team1 = tidy_team_name(alist[0].string)
                                        team1 = alist[0].string
                                        #if len(alist) > 2:
                                        #team2 = tidy_team_name(alist[2].string)
                                        team2 = alist[2].string
                                        #else:
                                        #    print(alist)
                                        bracketRow = [year, region, bracket_round, seed1, team1, currLoc, seed2, team2, winner_seed]
                                        #print(bracketRow)
                                        bracket.append(bracketRow)
                        bracket_round += 1
        sleep(0.5)
    return pd.DataFrame(bracket, columns=['Year', 'Region', 'Round', 'Seed', 'Team', 'Location', 'OpponentSeed', 'Opponent', 'WinningSeed']) 

    
df = get_bracket()
#df
df.to_csv('ncaa.csv')

In [170]:
df1 = df.loc[(df['Round'] == 1)]
fl_list = ['Florida', 'Florida State', 'Miami (FL)', 'UCF', 'Florida Gulf Coast', 'South Florida', 'Florida A&M;','Florida International', 'Florida Atlantic']

df1_teams_year = df1.loc[:, ['Year','Team','Opponent']]
df1_teams = df1.loc[:, ['Team','Opponent']]

fl_team_app = []
for fl_tm in fl_list:
    fl_tm_count = df1_teams[(df1_teams.Team == fl_tm) | (df1_teams.Opponent == fl_tm)]
    fl_team_app.append([fl_tm, len(fl_tm_count.index)])
sorted(fl_team_app, key=lambda l:l[1], reverse=True)

[['Florida', 18],
 ['Florida State', 14],
 ['Miami (FL)', 8],
 ['UCF', 3],
 ['South Florida', 3],
 ['Florida Gulf Coast', 2],
 ['Florida A&M;', 1],
 ['Florida International', 1],
 ['Florida Atlantic', 1]]

In [172]:
team_counts= df1_teams.stack().value_counts()

print(team_counts)
team_counts.to_csv('ncaa_team_appearances.csv')

Kansas                       37
UNC                          37
Kentucky                     36
Duke                         34
Villanova                    34
Syracuse                     34
Arizona                      33
Louisville                   32
UConn                        32
Notre Dame                   32
UCLA                         31
Texas                        31
Temple                       31
Indiana                      30
Marquette                    30
Oklahoma                     28
Illinois                     27
St. John's (NY)              27
West Virginia                26
Purdue                       26
Georgetown                   26
Michigan State               26
Pitt                         25
Arkansas                     25
BYU                          24
Xavier                       24
Princeton                    24
Ohio State                   24
Missouri                     24
Maryland                     23
                             ..
Texas-Ar

In [144]:
fl_teams_by_year = df1_teams_year[df1_teams_year.Team.isin(fl_list)|df1_teams_year.Opponent.isin(fl_list)] 
fl_teams_by_year

,Year,Team,Opponent
0,2016,UNC,Florida Gulf Coast
35,2016,Miami (FL),Buffalo
171,2014,Florida,Albany (NY)
196,2013,Miami (FL),Pacific
224,2013,Florida,Northwestern State
226,2013,Georgetown,Florida Gulf Coast
257,2012,Florida State,St. Bonaventure
269,2012,Temple,South Florida
303,2012,Florida,Virginia
337,2011,Florida,UCSB


In [145]:
grp_yr_count = fl_teams_by_year.groupby('Year')['Team'].count()
grp_yr_count

Year
1960    1
1968    1
1972    1
1978    1
1980    1
1987    1
1988    2
1989    2
1990    1
1991    1
1992    2
1993    1
1994    2
1995    2
1996    1
1998    2
1999    3
2000    2
2001    1
2002    3
2003    1
2005    2
2006    1
2007    1
2008    1
2009    1
2010    2
2011    2
2012    3
2013    3
2014    1
2016    2
Name: Team, dtype: int64